In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql.types import *
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as f
import time
import pyspark
import pandas as pd
import numpy as np

In [2]:
sc = SparkContext('local')
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

In [4]:
csv_list = ["09","10","11","12"]
df_list = []
activityList = [1001,1003,1004,1007,1008,1010,1099,1101,1102,1104,1105,1199,1201,1202,1203,1204,1205,1206,1207,1208,1299,2002,2003,2004,2099,2104,2107,2109,2110,2111,2112,2113,2199,2401,2402,2403,2404,2406,2407,2499,2299,2312,2317,4113,7104,7299,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6015,6016,6021,6099,6014,6019,6020,6018,6107,6113,6201,6202,6203,6204,6205,6206,6207,6208,6209,6210,6111,6112,6299,7101,7102,7103,7105,7106,7108,7199]

for listname in csv_list:
    df = sqlContext.read.format('com.databricks.spark.csv')\
        .options(header='true', inferSchema='true')\
        .load('C:/ssafy/2nd/s03p23a305/bigdata/analysis/2019-'+listname+'-UTF.csv')\
        .cache()
    
    # 기업 데이터 제거
    df = df.filter(df.개인기업구분 != "기업")
    df = df.filter(df.성별 != "0.기타")
    
    # 활동 분류
    df = df.filter(df.가맹점업종코드.isin(activityList))
    
    removeColList = ["회원_시도명", "회원_시군구명", "개인기업구분", "매출년월일", "결제금액", "결제건수"]
    for ls in removeColList:
        df = df.drop(ls)
    df.persist()
    df_list.append(df)

start = time.time()  # 시작 시간 저장

df_list[0].show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

In [206]:
df_list[0].take(3)

[Row(성별='1.남성', 연령='4.40대', 승인시간대1=13, 가맹점_시도명='강원', 가맹점_시군구명='강릉시', 가맹점_읍면동명='강동면', 가맹점업종코드=2004, 회원수=3),
 Row(성별='1.남성', 연령='4.40대', 승인시간대1=6, 가맹점_시도명='강원', 가맹점_시군구명='강릉시', 가맹점_읍면동명='강동면', 가맹점업종코드=2104, 회원수=3),
 Row(성별='1.남성', 연령='4.40대', 승인시간대1=7, 가맹점_시도명='강원', 가맹점_시군구명='강릉시', 가맹점_읍면동명='강동면', 가맹점업종코드=2104, 회원수=5)]

In [5]:
# schema = StructType([StructField("성별", StringType(), True), StructField("연령", StringType(), False),\
#                      StructField("승인시간대1", IntegerType(), True), StructField("가맹점업종코드", IntegerType(), True)])

# df_all = sqlContext.createDataFrame([],schema)
df_all = df_list[0]
# df_all.show()

In [6]:
for idx, df in enumerate(df_list):
    if(idx == 0):
        continue
    print(idx)
    df_all = df_all.union(df)

1
2
3


df_all.show()

In [7]:
df_all.colRegex

<bound method DataFrame.colRegex of DataFrame[성별: string, 연령: string, 승인시간대1: int, 가맹점_시도명: string, 가맹점_시군구명: string, 가맹점_읍면동명: string, 가맹점업종코드: int, 회원수: int]>

In [8]:
df_all_renamed = df_all.withColumnRenamed("성별", "gender")\
                        .withColumnRenamed("연령", "age")\
                        .withColumnRenamed("승인시간대1", "time")\
                        .withColumnRenamed("가맹점_시도명", "do")\
                        .withColumnRenamed("가맹점_시군구명", "si")\
                        .withColumnRenamed("가맹점_읍면동명", "dong")\
                        .withColumnRenamed("가맹점업종코드", "code")\
                        .withColumnRenamed("회원수", "n")                        

df_all_renamed.show()

In [9]:
rdd = df_all_renamed.rdd
rdd

MapPartitionsRDD[123] at javaToPython at NativeMethodAccessorImpl.java:0

In [10]:
rdd.take(3)

[Row(gender='1.남성', age='4.40대', time=13, do='강원', si='강릉시', dong='강동면', code=2004, n=3),
 Row(gender='1.남성', age='4.40대', time=6, do='강원', si='강릉시', dong='강동면', code=2104, n=3),
 Row(gender='1.남성', age='4.40대', time=7, do='강원', si='강릉시', dong='강동면', code=2104, n=5)]

In [11]:
rdd = rdd.map(lambda x: Row(gender=int(x[0][0]),\
                            age=int(x[1][0]),\
                            time=int(x[2]),\
                            do=x[3],\
                            si=x[4],\
                            dong = x[5],\
                            code =int(x[6]),\
                            n = int(x[7])))

In [12]:
rdd.take(3)[0]

Row(gender=1, age=4, time=13, do='강원', si='강릉시', dong='강동면', code=2004, n=3)

type(rdd)

In [13]:
schema = StructType([StructField("gender", StringType(), True),\
                     StructField("age", StringType(), True),\
                     StructField("time", IntegerType(), True),\
                     StructField("do", StringType(), True),\
                     StructField("si", StringType(), True),\
                     StructField("dong", StringType(), True),\
                     StructField("code", IntegerType(), True),\
                     StructField("n", IntegerType(), True)])


In [170]:
# df_mapped = spark.createDataFrame(rdd, schema).cache()
df_mapped = spark.createDataFrame(rdd, schema).persist(pyspark.StorageLevel.DISK_ONLY)

# 캐싱 없이 쇼
start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

# 캐싱 없이 재차 쇼
start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

# 캐싱 후 쇼
df_mapped.persist()
start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

df_mapped.unpersist()

In [171]:
start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

+------+---+----+----+------+------+----+---+
|gender|age|time|  do|    si|  dong|code|  n|
+------+---+----+----+------+------+----+---+
|     1|  4|  13|강원|강릉시|강동면|2004|  3|
|     1|  4|   6|강원|강릉시|강동면|2104|  3|
|     1|  4|   7|강원|강릉시|강동면|2104|  5|
|     1|  4|  11|강원|강릉시|강동면|2104|  3|
|     1|  4|   6|강원|강릉시|강동면|2104|  5|
|     1|  4|  17|강원|강릉시|강동면|2104|  3|
|     1|  4|   7|강원|강릉시|강동면|2104|  6|
|     1|  4|  11|강원|강릉시|강동면|2104|  3|
|     1|  4|   7|강원|강릉시|강동면|2104|  5|
|     1|  4|   6|강원|강릉시|강동면|2104|  5|
|     1|  4|   7|강원|강릉시|강동면|2104|  6|
|     1|  4|   7|강원|강릉시|강동면|2104|  4|
|     1|  4|   6|강원|강릉시|강동면|2104|  6|
|     1|  5|  11|강원|강릉시|강동면|2104|  3|
|     1|  4|   7|강원|강릉시|강동면|2104|  5|
|     1|  4|   6|강원|강릉시|강동면|2104|  3|
|     1|  4|  11|강원|강릉시|강동면|2104|  3|
|     1|  2|  11|강원|강릉시|강동면|2104|  3|
|     1|  4|   6|강원|강릉시|강동면|2104|  3|
|     1|  4|   7|강원|강릉시|강동면|2104|  7|
+------+---+----+----+------+------+----+---+
only showing top 20 rows

time : 8.24923038482666


In [172]:
df_mapped.registerTempTable("df_tmp")

df_mapped.count()

In [173]:
# 코드 형식
# df_mapped.groupBy('gender', 'age','time', 'code').count().select('gender','age','time', f.col('count').alias('n')).show()

# df_mapped.groupBy('gender', 'age','time', 'code').sum('n').show() # count().select('gender','age','time', f.col('count').alias('n')).show()
df_group = df_mapped.groupBy('gender', 'age','time', 'do', 'si','dong', 'code').sum('n')

# 쿼리 형식
df_group = sqlContext.sql("select *, sum(n) as cnt from df_tmp group by gender, age, time, code")

In [ ]:
#################
df_group = df_group.withColumnRenamed("sum(n)", "total").persist(pyspark.StorageLevel.DISK_ONLY)
df_group.show()
df_group.registerTempTable("df_group")


In [174]:
df_group = df_group.withColumnRenamed("sum(n)", "total").persist(pyspark.StorageLevel.DISK_ONLY)
df_group.columns

['gender', 'age', 'time', 'do', 'si', 'dong', 'code', 'total']

In [190]:
start = time.time()  # 시작 시간 저장

df_group.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

+------+---+----+----+------+------+----+-----+
|gender|age|time|  do|    si|  dong|code|total|
+------+---+----+----+------+------+----+-----+
|     2|  3|  15|강원|강릉시|견소동|2004|   60|
|     1|  4|  18|강원|강릉시|  교동|2002|    6|
|     2|  4|  14|강원|강릉시|  교동|2003|   24|
|     2|  3|   9|강원|강릉시|  교동|2003|   20|
|     1|  4|  14|강원|강릉시|  교동|2099|    9|
|     1|  5|  18|강원|강릉시|  교동|2104|  400|
|     1|  7|  13|강원|강릉시|  교동|2104|   12|
|     2|  4|  21|강원|강릉시|  교동|2104|   71|
|     1|  3|  13|강원|강릉시|  교동|2109|   12|
|     1|  5|  18|강원|강릉시|병산동|2104|    6|
|     2|  4|  16|강원|강릉시|성내동|2004|    6|
|     1|  4|  12|강원|강릉시|옥천동|2104|   78|
|     2|  3|  17|강원|강릉시|포남동|2104|    9|
|     1|  4|  15|강원|강릉시|홍제동|2004|   12|
|     1|  2|  19|강원|고성군|간성읍|2109|   25|
|     1|  4|  22|강원|동해시|나안동|2104|    3|
|     1|  4|  16|강원|동해시|천곡동|2003|    9|
|     1|  3|  12|강원|동해시|천곡동|2099|   39|
|     1|  3|  17|강원|동해시|천곡동|2099|   19|
|     2|  5|  12|강원|삼척시|남양동|2104|   32|
+------+---+----+----+------+------+----+-----+


# df_group.write.option("header", "true").csv('C:/ssafy/2nd/s03p23a305/bigdata/analysis/output/test')
df_group.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('C:/ssafy/2nd/s03p23a305/bigdata/analysis/output/output.csv')

In [191]:
df_group.registerTempTable("df_group")

In [192]:
si_df = df_group.select('si').distinct().persist(pyspark.StorageLevel.DISK_ONLY)
si_df.show()

+-------------+
|           si|
+-------------+
|       구리시|
|       강진군|
|       남동구|
|       송파구|
|       고령군|
|       시흥시|
|       도봉구|
|       부안군|
|       증평군|
|천안시 서북구|
|     북제주군|
|       김천시|
|       횡성군|
|       연기군|
|       영덕군|
|       상주시|
|       달성군|
|       양산시|
|       은평구|
|       순창군|
+-------------+
only showing top 20 rows



In [193]:
si_df.count()

241

In [194]:
241*35/60/60

2.343055555555556

In [195]:
si = "'종로구'"


In [196]:
query = "select gender, age, time, code, total from df_group where si ="+ si +" order by total desc"
query

"select gender, age, time, code, total from df_group where si ='종로구' order by total desc"

In [197]:
selected_df = sqlContext.sql(query).persist(pyspark.StorageLevel.DISK_ONLY)

start = time.time()  # 시작 시간 저장

selected_df.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

+------+---+----+----+-----+
|gender|age|time|code|total|
+------+---+----+----+-----+
|     2|  2|  18|6113| 4796|
|     2|  2|  17|6113| 4641|
|     2|  2|  19|6113| 4501|
|     2|  2|  16|6113| 4392|
|     1|  2|  17|6113| 4020|
|     1|  2|  16|6113| 3873|
|     1|  2|  18|6113| 3763|
|     1|  2|  15|6113| 3552|
|     1|  4|   8|2004| 3510|
|     2|  2|  15|6113| 3497|
|     1|  2|  19|6113| 3288|
|     2|  2|  20|6113| 3267|
|     2|  4|  15|6113| 3169|
|     2|  4|  16|6113| 3130|
|     2|  4|  17|6113| 3080|
|     1|  2|  18|2099| 3077|
|     2|  3|  16|6113| 2947|
|     2|  4|  14|6113| 2945|
|     2|  3|  15|6113| 2854|
|     1|  2|  14|6113| 2846|
+------+---+----+----+-----+
only showing top 20 rows

time : 0.0370023250579834


selected_df.count()

selected_df.persist()

In [198]:
selected_df.columns

['gender', 'age', 'time', 'code', 'total']

selected_df.describe()

selected_df.show()

In [199]:
selected_df.registerTempTable("selected_df")

In [200]:
query = "select code, sum(total) as sum from selected_df group by code order by sum desc"
query

'select code, sum(total) as sum from selected_df group by code order by sum desc'

In [201]:
sum_groupByDf = sqlContext.sql(query).persist(pyspark.StorageLevel.DISK_ONLY)


In [202]:
start = time.time()  # 시작 시간 저장

sum_groupByDf.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

+----+------+
|code|   sum|
+----+------+
|2104|422931|
|2004|323112|
|6113|246430|
|2199|199744|
|2099|154524|
|1004| 66570|
|2002| 40736|
|6018| 31848|
|2003| 28923|
|2112| 22970|
|2110| 16961|
|2107| 14821|
|2109| 10788|
|7106|  8718|
|2312|  2708|
|2499|  1852|
|6204|  1831|
|4113|  1231|
|2299|  1034|
|1201|   666|
+----+------+
only showing top 20 rows

time : 0.17597365379333496


total_sum = 0
for row in selected_df.collect():
    total_sum += row.total
total_sum

In [43]:
code_df = sqlContext.read.format('com.databricks.spark.csv')\
        .options(header='true', inferSchema='true')\
        .load('C:/ssafy/2nd/s03p23a305/bigdata/analysis/new_codelist-UTF.csv')\
        .cache()

In [45]:
code_df.collect()

[Row(largeCate='쇼핑', MediumCate='의류', smallCate='맞춤복점', code=1001, desc='남성복 맞춤업소,여성복 맞춤업소,와이셔츠 맞춤업소'),
 Row(largeCate='쇼핑', MediumCate='의류', smallCate='한복점', code=1003, desc='남.여 한복 맞춤업소, 개량한복점 포함'),
 Row(largeCate='쇼핑', MediumCate='의류', smallCate='기성복점', code=1004, desc='의류대리점(모피 및 교복 포함),남성기성복점,여성기성복점,와이셔츠판매점'),
 Row(largeCate='쇼핑', MediumCate='의류', smallCate='아동 및 유아복점', code=1007, desc='아동복 및 유아복 판매점'),
 Row(largeCate='쇼핑', MediumCate='의류', smallCate='내의판매점', code=1008, desc='내의,타월,양말,스타킹등 판매점'),
 Row(largeCate='쇼핑', MediumCate='의류', smallCate='양품점', code=1010, desc='각종 양품,잡화 판매점'),
 Row(largeCate='쇼핑', MediumCate='의류', smallCate='기타 의류', code=1099, desc=None),
 Row(largeCate='쇼핑', MediumCate='직물/침구류', smallCate='옷감판매점', code=1101, desc='양복,양장,한복등의 옷감 및 면직,견직,모직등의 직물류 판매점'),
 Row(largeCate='쇼핑', MediumCate='직물/침구류', smallCate='침구,커튼,카펫트점', code=1102, desc='이불,담요,베개,카바,커튼,카펫트등 판매점'),
 Row(largeCate='쇼핑', MediumCate='직물/침구류', smallCate='수예점', code=1104, desc='각종 수예품(자수,실포함) 판매점'),
 